# Feature Scalings

Create a dummy dataset with large range difference in values across features. Apply feature scaling methods such as MinMax scaling and Standard scaling and compare. Save the transform model to featureScaling.mlnet. Once completed, print 'task resolved' in the end.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

// Define the data schema
public class InputData
{
    [LoadColumn(0)] public float Feature1;
    [LoadColumn(1)] public float Feature2;
}

public class TransformedData
{
    public float NormalizedFeature1;
    public float NormalizedFeature2;
    public float StandardizedFeature1;
    public float StandardizedFeature2;
}

// Create a dummy dataset
var data = new List<InputData>
{
    new InputData { Feature1 = 1, Feature2 = 100 },
    new InputData { Feature1 = 2, Feature2 = 200 },
    new InputData { Feature1 = 3, Feature2 = 300 },
    new InputData { Feature1 = 4, Feature2 = 400 },
    new InputData { Feature1 = 5, Feature2 = 500 }
};

// Create a new MLContext
var context = new MLContext();

// Load the data into IDataView
var dataView = context.Data.LoadFromEnumerable(data);

// Define the pipeline
var pipeline = context.Transforms.NormalizeMinMax("NormalizedFeature1", "Feature1")
    .Append(context.Transforms.NormalizeMinMax("NormalizedFeature2", "Feature2"))
    .Append(context.Transforms.NormalizeMeanVariance("StandardizedFeature1", "Feature1"))
    .Append(context.Transforms.NormalizeMeanVariance("StandardizedFeature2", "Feature2"));

// Fit the pipeline
var transformer = pipeline.Fit(dataView);

// Transform the data
var transformedData = transformer.Transform(dataView);

// Save the transform model
context.Model.Save(transformer, dataView.Schema, "featureScaling.mlnet");

// Print the transformed data
var transformedDataList = context.Data.CreateEnumerable<TransformedData>(transformedData, reuseRowObject: false).ToList();
foreach (var item in transformedDataList)
{
    Console.WriteLine($"NormalizedFeature1: {item.NormalizedFeature1}, NormalizedFeature2: {item.NormalizedFeature2}, StandardizedFeature1: {item.StandardizedFeature1}, StandardizedFeature2: {item.StandardizedFeature2}");
}

Console.WriteLine("task resolved");